In [21]:
from Node import Node
from Object import Rect,Point
from graphviz import Digraph
import math
import random
import vtk
import time

dots=[]
actors=[]
fond = []

class Rect:
    def __init__(self,x1,y1,x2,y2):
        self.x1=x1
        self.x2=x2
        self.y1=y1
        self.y2=y2

    def perimeter(self):
        return 2*(abs(self.x2-self.x1)\
                +abs(self.y2-self.y1))

    def contain_rect(self,rect):
        return self.x1<rect.x1 and self.y1<rect.y1 and \
                self.x2<rect.x2 and self.y2<rect.y2

    def __str__(self):
        return "MBR: ({}, {}), ({}, {})".format(self.x1, self.y1, self.x2, self.y2)

    def intersect(self,rect):
        if (self.y1>rect.y2 or self.y2<rect.y1 or
                self.x1>rect.x2 or self.x2<rect.x1):
            return False
        return True

    def is_covered(self,point):
        return self.x1<=point.x<=self.x2 and self.y1<=point.y<=self.y2

class Point:
    def __init__(self,id1,x,y):
        self.id=id1
        self.x=x
        self.y=y

    def __str__(self):
        return "({}, {})".format(self.x, self.y)



def rectVtk(x,y,z,w,h,p):
    cube=vtk.vtkCubeSource()
    cube.SetXLength(w)
    cube.SetYLength(h)
    cube.SetZLength(p)
    cube.SetCenter(x,y,z)
    cubeMapper=vtk.vtkPolyDataMapper()
    cubeMapper.SetInputConnection(cube.GetOutputPort())
    cubeActor=vtk.vtkActor()
    cubeActor.GetProperty().SetColor(x,y,2)
    cubeActor.GetProperty().SetOpacity(0.2)
    cubeActor.SetMapper(cubeMapper)
    fond.append(cubeActor)
    
def planeVtk(x,y,w,h,R=0,G=0,B=0):
    cube=vtk.vtkCubeSource()
    cube.SetXLength(w)
    cube.SetYLength(h)
    cube.SetZLength(0)
    cube.SetCenter(x,y,0)
    cubeMapper=vtk.vtkPolyDataMapper()
    cubeMapper.SetInputConnection(cube.GetOutputPort())
    cubeActor=vtk.vtkActor()
    cubeActor.GetProperty().SetColor(x+R,y+G,w+B)
    cubeActor.GetProperty().SetOpacity(0.2)
    cubeActor.SetMapper(cubeMapper)
    actors.append(cubeActor)    

def pointVtk(point,R=0):
    sphere=vtk.vtkSphereSource()
    sphere.SetRadius(3.5)
    sphere.SetCenter(point[0],point[1],point[2])
        
    sphereMapper=vtk.vtkPolyDataMapper()
    sphereMapper.SetInputConnection(sphere.GetOutputPort())
    sphereActor=vtk.vtkActor()
    sphereActor.GetProperty().SetColor(R,0,255)
    sphereActor.SetMapper(sphereMapper)
    dots.append(sphereActor)

class Node:
    def __init__(self,B):
        self.B = B
        self.child_nodes = []
        self.points = []
        self.parent_node = None
        self.MBR = Rect(-1,-1,-1,-1)

    def getObject(self):
        return self.points if self.is_leaf() else self.MBR

    def show(self,dot):
        if(self.is_leaf()):
            v=[]
            for point in self.points:
                v.append(str(point))
                pointVtk((point.x,point.y,0))
                print(str(point))
            dot.node(str(self),str(v))
            return v
        else:
            dot.node(str(self),str(self.MBR))
            planeVtk(self.MBR.x1+self.MBR.x2/2,self.MBR.y1+self.MBR.y2/2,self.MBR.x2,self.MBR.y2)
            print(str(self.MBR))
            for child in self.child_nodes:
                data=child.show(dot)
                
                dot.edge(str(self),str(child))
                
            return self.MBR

    def get_parent(self):
        return self.parent_node

    def insert_point(self,point):
        self.points.append(point)

    def is_leaf(self):
        return len(self.child_nodes) == 0

    def is_overflow(self):
        return (self.is_leaf() and len(self.points) > self.B ) or \
                (not self.is_leaf() and len(self.child_nodes) > self.B)

    def insert_child_node(self,node):
        node.parent_node=self
        self.child_nodes.append(node)

    def updateMBR(self):
        if(self.is_leaf()):
            datax=[point.x for point in self.points]
            datay=[point.y for point in self.points]
            self.MBR.x1=min(datax)
            self.MBR.x2=max(datax)
            self.MBR.y1=min(datay)
            self.MBR.y2=max(datay)

        else:
            datax=[child.MBR.x1 for child in self.child_nodes]
            datax1=[child.MBR.x2 for child in self.child_nodes]
            datay=[child.MBR.y1 for child in self.child_nodes]
            datay1=[child.MBR.y2 for child in self.child_nodes]
            self.MBR.x1=min(datax)
            self.MBR.x2=max(datax1)
            self.MBR.y1=min(datay)
            self.MBR.y2=max(datay1)

        if( self.parent_node and not self.parent_node.MBR.contain_rect(self.MBR)):
            self.parent_node.updateMBR()

    def get_pointsMBR_perimeter(self,points):
        x1=min([point.x for point in points ])
        x2=max([point.x for point in points ])
        y1=min([point.y for point in points ])
        y2=min([point.y for point in points ])
        return Rect(x1,y1,x2,y2).perimeter()

    def get_nodesMBR_perimeter(self,nodes):

        x1=min([node.MBR.x1 for node in nodes ])
        x2=max([node.MBR.x2 for node in nodes ])
        y1=min([node.MBR.y1 for node in nodes ])
        y2=max([node.MBR.y2 for node in nodes ])
        return Rect(x1,x2,y1,y2).perimeter()

    def perimeter(self):
        return self.MBR.perimeter()

    def perimeter_increase_with_point(self,point):
        x1=point.x if point.x < self.MBR.x1 else self.MBR.x1
        y1=point.y if point.y < self.MBR.y1 else self.MBR.y1
        x2=point.x if point.x > self.MBR.x2 else self.MBR.x2
        y2=point.y if point.y > self.MBR.y2 else self.MBR.y2
        return Rect(x1,y1,x2,y2).perimeter()-self.perimeter()


points_interset=[]


def range_query(region,node):
    
    if(node.is_leaf()):
        for point in node.points:
            if(region.is_covered(point)):
                points_interset.append(point)
        #return points

    else:
        #points=[]
        for child in node.child_nodes:
            if( region.intersect(child.MBR)):
                range_query(region,child)
                #points+=range_query(region,child)

        #return points

class RTree:
    def __init__(self, B=4):
        self.B = B
        self.root = Node(self.B) 

    def insert(self,point,node=None):
        if(node is None):
            node = self.root

        if(node.is_leaf()):
            node.insert_point(point)
            #node.updateMBR()
            if (node.is_overflow()):
                self.handle_overflow(node)

        else:
            node_v=self.choose_subtree(node,point)
            self.insert(point,node_v)

    def handle_overflow(self,node):
        #split
        node,new_node = self.split_leaf_node(node) if node.is_leaf() else self.split_internal_node(node)

        if (node is self.root) :
            self.root = Node(self.B)
            self.root.insert_child_node(node)
            self.root.insert_child_node(new_node)
            self.root.updateMBR()
        else:
            
            w_node=node.get_parent()
            node.updateMBR()
            w_node.insert_child_node(new_node)
            #w_node.updateMBR()
            if(w_node.is_overflow()):
                    self.handle_overflow(w_node)


    def choose_subtree(self,node,point):
        best_child=None
        best_perimeter = 0

        for item in node.child_nodes:
            if( node.child_nodes.index(item) == 0 or \
                best_perimeter > item.perimeter_increase_with_point(point)):
                best_child=item
                best_perimeter=item.perimeter_increase_with_point(point)
        return best_child

    def split_leaf_node(self,node):
        m = len(node.points)
        best_perimeter = -1
        best_set1 = []
        best_set2 = []
        pointsortx=sorted(node.points, key=lambda point: point.x)
        for i in range(int(0.4*m),int(0.6*m)+1):
            S1=pointsortx[:i]
            S2=pointsortx[i:]
            tempP = node.get_pointsMBR_perimeter(S1)\
                    +node.get_pointsMBR_perimeter(S2)
            if( best_perimeter == -1 or best_perimeter > tempP):
                best_perimeter = tempP
                best_set1=S1
                best_set2=S2

        pointsorty=sorted(node.points, key=lambda point: point.y)
        
        for i in range(int(0.4*m),int(0.6*m)+1):
            S1=pointsorty[:i]
            S2=pointsorty[i:]
            tempP=node.get_pointsMBR_perimeter(S1)\
                    +node.get_pointsMBR_perimeter(S2)
            if( best_perimeter == -1 or best_perimeter > tempP):
                best_perimeter = tempP
                best_set1=S1
                best_set2=S2
        
        node.points=best_set1
        node.updateMBR()
        new_node=Node(self.B)
        for pointt in best_set2:
            new_node.insert_point(pointt)
        new_node.updateMBR()
        return node,new_node
        
    def split_internal_node(self,node):
        m = len(node.child_nodes)
        best_perimeter = -1
        best_set1 = []
        best_set2 = []
        pointsortx1=sorted(node.child_nodes, key=lambda child: child.MBR.x1)
        for i in range(int(0.4*m),int(0.6*m)+1):
        
            S1=pointsortx1[:i]
            S2=pointsortx1[i:]
            tempP = node.get_nodesMBR_perimeter(S1)\
                    +node.get_nodesMBR_perimeter(S2)
            if( best_perimeter == -1 or best_perimeter > tempP):
                best_perimeter = tempP
                best_set1=S1
                best_set2=S2
        
        pointsortx2=sorted(node.child_nodes, key=lambda child: child.MBR.x2)
        for i in range(int(0.4*m),int(0.6*m)+1):
        
            S1=pointsortx2[:i]
            S2=pointsortx2[i:]
            tempP = node.get_nodesMBR_perimeter(S1)\
                    +node.get_nodesMBR_perimeter(S2)
            if( best_perimeter == -1 or best_perimeter > tempP):
                best_perimeter = tempP
                best_set1=S1
                best_set2=S2

        pointsorty1=sorted(node.child_nodes, key=lambda child: child.MBR.y1)
        for i in range(int(0.4*m),int(0.6*m)+1):
            S1=pointsorty1[:i]
            S2=pointsorty1[i:]
            tempP=node.get_nodesMBR_perimeter(S1)\
                    +node.get_nodesMBR_perimeter(S2)
            if( best_perimeter == -1 or best_perimeter > tempP):
                best_perimeter = tempP
                best_set1=S1
                best_set2=S2
        pointsorty2=sorted(node.child_nodes, key=lambda child: child.MBR.y2)
        
        for i in range(int(0.4*m),int(0.6*m)+1):
            S1=pointsorty2[:i]
            S2=pointsorty2[i:]
            tempP=node.get_nodesMBR_perimeter(S1)\
                    +node.get_nodesMBR_perimeter(S2)
            if( best_perimeter == -1 or best_perimeter > tempP):
                best_perimeter = tempP
                best_set1=S1
                best_set2=S2
        
        node.child_nodes =best_set1
        node.updateMBR()
        new_node=Node(self.B)
        for pointt in best_set2:
            new_node.insert_child_node(pointt)
        new_node.updateMBR()
        return node,new_node

    

    def show(self):
        dot=Digraph()
        self.root.show(dot)
        #print(dot.source)
        #dot.render('R-tree.gv',view=True)
        
        query = Rect(random.randint(0,200),random.randint(0,200),random.randint(0,200),random.randint(0,200))
        range_query(query,self.root)
        planeVtk(query.x1,query.y1,query.x2,query.y2,255)
        
        
        ren = vtk.vtkRenderer()
        rectVtk (200,200,0,400,400,0)
        ren.AddActor(fond[0])
        renWin = vtk.vtkRenderWindow()
        renWin.AddRenderer(ren)
        renWin.SetSize(600, 600)
        iren = vtk.vtkRenderWindowInteractor()
        iren.SetRenderWindow(renWin)
#        ren.SetBackground(255,255,255)
        print("QUery")
        for  i in points_interset:
            print(str(i))
            pointVtk((i.x,i.y,0))
        for act in dots:
            ren.AddActor(act)
            renWin.AddRenderer(ren)
            iren.SetRenderWindow(renWin)
            renWin.Render()
            #time.sleep(0.3)
        for act in actors:
            ren.AddActor(act)
            renWin.AddRenderer(ren)
            iren.SetRenderWindow(renWin)
            renWin.Render()
            time.sleep(0.3)

        iren.Start()
  
    
def main():
    tree=RTree(4)
    for i in range(15):
        tree.insert(Point(i,random.randint(0,200),random.randint(0,200)))

    tree.show()

main()


MBR: (20, 11), (183, 151)
MBR: (53, 11), (134, 124)
(131, 11)
(134, 35)
(88, 24)
(104, 51)
(119, 55)
(53, 43)
(57, 124)
MBR: (20, 42), (183, 151)
(154, 42)
(162, 135)
(183, 102)
(182, 34)
(20, 151)
(37, 136)
(15, 138)
(19, 172)
QUery
